In [1]:
import requests
from bs4 import BeautifulSoup
import xlrd
import pandas as pd
import unidecode
import numpy as np


xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True


In [34]:
def get_valid_columns_df(df):
    """
    """
    null_count_per_column = df.replace('',np.nan).isnull().sum(axis=0)
    valid_columns = null_count_per_column/len(df)  >= 0.9
    valid_columns  = valid_columns[~valid_columns].index
    inter_df = df[valid_columns]
    
    inter_df = inter_df.T
    
    null_count_per_column = inter_df.replace('', np.nan).isnull().sum(axis=0)
    valid_columns = null_count_per_column/len(inter_df)  >= 0.5
    valid_columns  = valid_columns[~valid_columns].index
    
    
    df = inter_df[valid_columns].T
    
    columns = list(df.iloc[0].values)
    
    count_columns = {}
    columns_unique = []
    
    for item in columns:
        if item not in count_columns:
            count_columns[item] = 1
        else:
            count_columns[item] += 1
        if count_columns[item] > 1:
            item +='__' + str(count_columns[item])
        columns_unique.append(item)
        
         
    df = df.iloc[1:]
    
    df.columns = columns_unique
    
    return df


def clean_text(txt):
    """
    """
    txt = txt.lower()
    txt = unidecode.unidecode(txt)
    txt = txt.replace('\n', ' ')
    txt = txt.replace('  ', ' ')
    txt = txt.strip(' ')
    txt = txt.replace(' ', '_')
    return txt

def clean_dataframe(df):
    """
    """
    df = get_valid_columns_df(df)
    #df.columns = new_columns.apply(clean_text)
    nan_in_cols = df.replace('', None).isna().sum()
    cols = nan_in_cols[nan_in_cols <= 10*nan_in_cols.median()].index
    df = df[cols]
    nan_in_rows = df.replace('', None).T.isna().sum()
    rows = nan_in_rows[nan_in_rows <= 10*nan_in_rows.median()].index
    df = df.loc[rows].reset_index(drop=True)
    df = df.iloc[1:].reset_index(drop=True)
    return df

def read_workbook(path):
    """
    """
    xl_info = requests.get(path)
    workbook = xlrd.open_workbook(file_contents=xl_info.content)
    return workbook

In [3]:
# URL deseada para obtener la información:
url = 'https://www.gob.mx/cnbv/acciones-y-programas/bases-de-datos-de-inclusion-financiera'
# Obtener una respuesta.
html = requests.get(url)
if not html.ok:
    print('Response is between 400 and 600.', html.reason)
# Obtener el contenido
html_content = html.content
# Hacer una sopa bonita para buscar contenido de manera sencilla
soup = BeautifulSoup(html_content)

link_list = []
# En este caso los hipervínculos de interés están en a
for a_element in soup.find_all('a'):
    try:
        'Agarramos esos hipervínculos'
        link = a_element.__getitem__('href')
        link_list.append(link)
    except KeyError:
        print('element does not contain href')
        
excel_links = [excel_dataset_link for excel_dataset_link in link_list if 'xls' in excel_dataset_link]

element does not contain href
element does not contain href
element does not contain href


In [37]:
clean_info = []
raw_info = []
clean_dict = {}
raw_dict = {}
not_info_as_expected = []
for excel_path in excel_links:
    workbook = read_workbook(excel_path)
    workbook_sheet_names = workbook.sheet_names()
    desired_sheet_set = [mun_sheet for mun_sheet in workbook_sheet_names if 'Mun' in mun_sheet]
    for desired_sheet in desired_sheet_set:
        if desired_sheet:
            df = pd.DataFrame(workbook.sheet_by_name(desired_sheet)._cell_values)
            raw_info.append(df)
            df_clean = clean_dataframe(df.copy())
            
            name = excel_path.split('/')[-1]
            date = name.replace('%20', '_').split('_')[-1]
            date = date.split('.')[0]
            df_clean['name'] = name
            df_clean['sheet_name'] = desired_sheet
            df_clean['url'] = excel_path
            df_clean['date'] = date
            
            df['name'] = name
            df['sheet_name'] = desired_sheet
            df['url'] = excel_path
            df['date'] = date
            
            clean_info.append(df_clean)
            if clean_dict.get(desired_sheet) is None:
                clean_dict[desired_sheet] = []
                raw_dict[desired_sheet] = []
                
            clean_dict[desired_sheet].append(df_clean)
            raw_dict[desired_sheet].append(df)
            print(name, desired_sheet, sep='|')
        else:
            not_info_as_expected.append(excel_path)

Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Tenencia Uso Banca Mun
Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Tenencia Uso EACP Mun
Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Género Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Tenencia Uso Banca Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Tenencia Uso EACP Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Género Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Tenencia Uso Banca Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Tenencia Uso EACP Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Género Mun
Base_de_Datos_de_Inclusion_Financiera_202009.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202009.xlsm|BD Tenencia U

Base%20de%20Datos%20IF%20201109.xls|BD Uso Mun
Base%20de%20Datos%20IF%20201109.xls|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201106.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201106.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201106.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201103.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201103.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201103.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201012.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201012.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201012.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201009.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201009.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201009.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201006.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201006.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201003.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201003.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20200912.xlsx|Municipios


In [39]:
clean_dict.keys()

dict_keys(['BD Infraestructura Mun', 'BD Tenencia Uso Banca Mun', 'BD Tenencia Uso EACP Mun', 'BD Género Mun', 'BD Acceso Mun', 'BD Uso Banca Mun', 'BD Uso EACP Mun', 'BD Uso Mun', 'Municipios'])

In [42]:
for item in clean_dict['BD Uso Mun']:
    print(item.columns.values,'\n')

['Clave Municipio' 'Clave \nEstado' 'Región' 'Estado' 'Municipio'
 'Superficie km2' 'Población' 'Población adulta' 'Tipo de población'
 'Cuentas transaccionales nivel 1' 'Cuentas transaccionales nivel 2'
 'Cuentas transaccionales nivel 3' 'Cuentas transaccionales tradicionales'
 'Cuentas de ahorro' 'Depósitos a plazo' 'Tarjetas de débito'
 'Tarjetas de Crédito' 'Hipotecario ' 'Grupal' 'Personal' 'Nómina'
 'Automotriz' 'ABCD' 'En TPV' 'En cajeros automáticos'
 'Cuentas transaccionales nivel 1__2' 'Cuentas transaccionales nivel 2__2'
 'Cuentas transaccionales nivel 3__2'
 'Cuentas transaccionales tradicionales__2' 'Cuentas de ahorro__2'
 'Depósitos a plazo__2' 'Tarjetas de débito__2' 'Tarjetas de Crédito__2'
 'Hipotecario __2' 'Grupal__2' 'Personal__2' 'Nómina__2' 'Automotriz__2'
 'ABCD__2' 'En TPV__2' 'En cajeros automáticos__2' 'name' 'sheet_name'
 'url' 'date'] 

['Clave Municipio' 'Clave \nEstado' 'Región' 'Estado' 'Municipio'
 'Superficie km2' 'Población' 'Población adulta' 'Tipo de

In [ ]:
X = raw_dict['BD Acceso Mun'][4].drop(columns=['name','date','sheet_name','url']).copy()

Y = raw_dict['BD Acceso Mun'][0].drop(columns=['name','date','sheet_name','url']).copy()



clean_dataframe(X).columns.values


In [ ]:
clean_dict['BD Acceso Mun'][4]

In [ ]:
clean_dict.keys()

In [ ]:
bd_infraestructura_mun = pd.concat(clean_dict['BD Infraestructura Mun'])

In [ ]:
bd_tendencia_uso_banca_mun = pd.concat(clean_dict['BD Tenencia Uso Banca Mun'])

In [ ]:
bd_tendencia_uso_eacp_mun = pd.concat(clean_dict['BD Tenencia Uso EACP Mun'])

In [ ]:
bd_genero_mun = pd.concat(clean_dict['BD Género Mun'])

In [ ]:
def get_unique_id(df):
    clave = df.informacion_general_clave_municipio.astype('int').astype('str')
    fecha = df.date.astype('str').apply(lambda x: x)
    df['clave_unica'] = clave + '_' + fecha
    return df

In [ ]:
bd_tendencia_uso_banca_mun = get_unique_id(bd_tendencia_uso_banca_mun)
bd_tendencia_uso_eacp_mun = get_unique_id(bd_tendencia_uso_eacp_mun)
bd_infraestructura_mun = get_unique_id(bd_infraestructura_mun)
bd_genero_mun = get_unique_id(bd_genero_mun)

In [ ]:
nan_gstuff = bd_genero_mun[bd_genero_mun.date.isin(['202103', '202003', '201903'])].isna().sum()
non_write_colsg = nan_gstuff[nan_gstuff>1].index
simple_bd_genero_mun = bd_genero_mun.drop(non_write_colsg, axis=1)

In [ ]:
drop_cols = ['date',
 'indicadores_(transacciones_por_cada_10,000_adultos)_transacciones_en_cajeros_automaticos',
 'indicadores_(transacciones_por_cada_10,000_adultos)_transacciones_en_tpv',
 'indicadores_demograficos_(contratos_por_cada_10,000_adultos)_captacion_depositos_a_plazo',
 'indicadores_demograficos_(contratos_por_cada_10,000_adultos)_captacion_tarjetas_de_debito',
 'informacion_general_clave_estado',
 'informacion_general_clave_municipio',
 'informacion_general_estado',
 'informacion_general_municipio',
 'informacion_general_poblacion',
 'informacion_general_poblacion_adulta',
 'informacion_general_poblacion_adulta_hombres',
 'informacion_general_poblacion_adulta_mujeres',
 'informacion_general_region',
 'informacion_general_tipo_de_poblacion',
 'name',
 'sheet_name',
 'url']

drop_cols2 = [ 'date',
 'informacion_general_clave_estado',
 'informacion_general_clave_municipio',
 'informacion_general_estado',
 'informacion_general_municipio',
 'informacion_general_poblacion',
 'informacion_general_poblacion_adulta',
 'informacion_general_poblacion_adulta_hombres',
 'informacion_general_poblacion_adulta_mujeres',
 'informacion_general_region',
 'informacion_general_tipo_de_poblacion',
 'name',
 'sheet_name',
 'url']

In [ ]:
general_data = bd_tendencia_uso_banca_mun.merge(
    bd_tendencia_uso_eacp_mun.drop(drop_cols, axis=1), on='clave_unica', how='left'
).merge(
    bd_infraestructura_mun.drop(drop_cols2, axis=1), on='clave_unica', how='left'
)
nan_stuff = general_data.isna().sum()
non_write_cols = nan_stuff[nan_stuff>1].index
general_data = general_data.drop(non_write_cols, axis=1)
simplified_data = general_data[general_data.clave_unica.str.endswith('03')]

In [ ]:
drop_cols3 =[ 'date',
 'informacion_general_clave_estado',
 'informacion_general_clave_municipio',
 'informacion_general_estado',
 'informacion_general_municipio',
 'informacion_general_poblacion',
 'informacion_general_poblacion_adulta',
 'informacion_general_poblacion_adulta_hombres',
 'informacion_general_poblacion_adulta_mujeres',
 'informacion_general_region',
 'informacion_general_tipo_de_poblacion',
 'name',
 'sheet_name',
 'url'
]
simplified_data2 = simplified_data.merge(
    simple_bd_genero_mun.drop(drop_cols3, axis=1), 
    on='clave_unica',
    how='left')

In [ ]:
simplified_data2.to_csv('./../rulo_mota_omath/data/raw/general_financial_inclusion.csv')